# Day 21

https://adventofcode.com/2022/day/21

In [1]:
def parse21(filename):
    with open(filename) as f:
        monkeyOps = {}
        for k,v in [ l.strip().split(": ") for l in f.readlines() ]:
            if v.isdigit():
                monkeyOps[k] = int(v)
            else:
                o = v.split(" ")
                monkeyOps[k] = (o[0],o[1],o[2]) 
        return monkeyOps

In [2]:
def monkeyValue(name,monkeyOps):
    if type(monkeyOps[name])==int:
        return monkeyOps[name]
    else:
        o1,op,o2 = monkeyOps[name]
        a = monkeyValue(o1,monkeyOps)
        b = monkeyValue(o2,monkeyOps)
        r = 0
        if op=="+":
            r = a+b
        elif op=="-":
            r = a-b
        elif op=="*":
            r = a*b
        elif op=="/":
            r = a/b
        monkeyOps[name] = r
        return r

In [3]:
monkeyOps0 = parse21("examples/example21.txt")
monkeyOps = parse21("AOC2022inputs/input21.txt")

print("Test 1:",int(monkeyValue('root',monkeyOps0)))
print("Part 2:",int(monkeyValue('root',monkeyOps)))

Test 1: 152
Part 2: 83056452926300


## Part 2

Observations:

- The second term of the `root` expression does not change, I only need to calculate the first one
- Standard brute force for the full example woiuld take too long...
- ... but I can use a iterative/dicotomic approach to quickly converge to input `humn` value, noting that increasing `humn` reduces the first term in the full input. 
- I can exit the dicotomic search when I'm close enough, and run a brute-force scan aroung the minumum

In [4]:
from copy import deepcopy

def monkeyRoot(humn,monkeyOps):
    monkeyOpsCalc = deepcopy(monkeyOps)
    monkeyOpsCalc['humn'] = humn
    o1,_,o2 = monkeyOpsCalc['root']
    return monkeyValue(o1,monkeyOpsCalc), monkeyValue(o2,monkeyOpsCalc)

In [5]:
monkeyOps0 = parse21("examples/example21.txt")

monkeyRoot(301,monkeyOps0)

(150.0, 150)

In [6]:
monkeyOps0 = parse21("examples/example21.txt")

h = 290
while True:
    o1,o2 = monkeyRoot(h,monkeyOps0)
    print(h,o1,o2)
    if o1==o2:
        print(h,o1,o2)
        break
    h+=1

290 144.5 150
291 145.0 150
292 145.5 150
293 146.0 150
294 146.5 150
295 147.0 150
296 147.5 150
297 148.0 150
298 148.5 150
299 149.0 150
300 149.5 150
301 150.0 150
301 150.0 150


In [7]:
def part2(monkeyOps,verbose=False):
    h = 0
    i = 0
    # "dicotomic" search, stop if I'm close enough
    while True:
        h = int(h)
        o1,o2 = monkeyRoot(h,monkeyOps)
        if verbose:
            print(h,o1,o2,o1-o2)
        if o1==o2:
            break
        h += abs(o1-o2)//10
        i += 1
        if abs(o1-o2)<100:
            break
    # direct scan around solution
    h = int(h)
    while True:
        o1,o2 = monkeyRoot(h,monkeyOps)
        if verbose:
            print(h,o1,o2)
        if o1==o2:
            break
        h+=1
    return h

In [8]:
monkeyOps = parse21("AOC2022inputs/input21.txt")

part2(monkeyOps)

3469704905529

In [9]:
monkeyOps0 = parse21("examples/example21.txt")

part2(monkeyOps0)

301

## Semi analytical approach to Part 2

In [10]:
def monkeyExpression(k,monkeyOps):
    '''Compile expression for variable k'''
    if k in ['humn','+','-','*',"/"]:
        return k
    if type(monkeyOps[k])==int:
        return str(monkeyOps[k])
    expr = "(" + "".join([ monkeyExpression(o,monkeyOps) for o in monkeyOps[k] ]) + ")"
    monkeyOps[k] = expr
    return expr

### Using `sympy` to simplify expressions...

In [11]:
import sympy
from sympy.parsing.sympy_parser import *

monkeyOpsExpr = deepcopy(monkeyOps)

o1,_,o2 = monkeyOpsExpr['root']
f1 = monkeyExpression(o1,monkeyOpsExpr)
f2 = monkeyExpression(o2,monkeyOpsExpr)

f2 = int(eval(f2)) # this is a constant!

f1 = f1.replace('humn','x')
x = sympy.symbols("x")
y = sympy.symbols("y")
f = parse_expr(f1)
func = sympy.Eq(y,f)

print(func)

Eq(y, 2640586771140514/45 - 4004*x/405)


In [12]:
def y(x):
    a = 2640586771140514
    b = 45
    c = 4004
    d = 405
    return a/b - c*x/d

In [13]:
sol1 = int( y( monkeyOps['humn'] ) + f2 )
print("Part 1:",sol1)

Part 1: 83056452926300


In [16]:
def x(y):
    '''Inverse function of first root term'''
    a = 2640586771140514
    b = 45
    c = 4004
    d = 405
    return -d/c*(y - a/b)

In [17]:
sol2 = int(x(f2))
print("Part 2:",sol2)

Part 2: 3469704905529
